In [11]:
import pandas as pd
import sqlite3,re,hashlib,datetime

In [12]:
def hashmaker(text):
    return hashlib.md5((text).encode('utf-8')).hexdigest()

def dptfinder(text):
    return text[0:2]

def surfacefinder(text):
    rex = re.compile(r'.*?,(.*?)m2.*?',re.S|re.M)
    matc = rex.match(str(text))
    if matc:
        return(matc.groups()[0].strip())
    else:
        return("0")

def find_type(text): # tries les annonces: 0 si rien, 1 si appartement, 2 si maison, 3 si terrain
    x=0 #si il n'y a rien dans l'annonce, ce n'est pas classé
    if "terrain" in text:
        x=3
    if "Terrain" in text:
        x=3
    if "Appartement" in text:
        x=1
    if "appartement" in text:
        x=1
    if "Studio" in text:
        x=1
    if "studio" in text:
        x=1
    if "maison" in text:
        x=2
    if "Maison" in text:
        x=2
    if "propriété" in text:
        x=2
    if "Propriété" in text:
        x=2
    return x

def itemizer(df):
    temp=df[df["hash"].duplicated(keep=False)]  # on drop  les items qui sont uniques , car on est la pour du suivi de changements
    return(temp.groupby('hash').combinaison.apply(list).reset_index()) #on groupe par hash
  

In [13]:
conn = sqlite3.connect("toutf.db")
df2 = pd.read_sql_query("select * from data;", conn)
conn = sqlite3.connect("toutff.db")
df1 = pd.read_sql_query("select * from data;", conn)

df=pd.concat([df1, df2]) 

In [14]:
#on transforme les colonnes en types exploitables
df['prix']=df['prix'].astype(float)
df['id']=df['id'].astype(float)
df['taux']=df['taux'].astype(float)
df['date'] = df['date'].astype('datetime64[ns]')

#on catégorise l'appart
df['type'] = df['nom'].apply(find_type)

#on fait un hash
df['hash'] = df['lien'].apply(hashmaker)

#on trouve les surfaces
df['surface'] = df['nom'].apply(surfacefinder)

#on trouve les  departements
df['dept'] = df['codepostal'].apply(dptfinder)
df['dept']= df['dept'].astype(str)



In [22]:
#test

xf=df[df.date > datetime.datetime.now() - pd.to_timedelta("60day")]
#df.tail(10)
fdf = xf.sort_values(by=['prix'], ascending=False)
fdf.tail(30)
#test

,id,taux,nom,prix,date,lien,type,surface,codepostal,hash,dept
21584,21586.0,-19.05,13140 Miramas - Vends,8500.0,2020-05-21,"/miramas,d81031957",0,0,13140,2952a484cf345ac0bc0303d9ead87187,13
15385,15387.0,-18.05,22160 Calanhel - Terrain,8479.0,2020-05-18,"/calanhel,d71733888",3,0,22160,42201e1866cc4bcc40c551bad1b52024,22
20997,20999.0,-7.53,"36800 Saint-gaultier - Terrain, 515m2",8189.0,2020-05-21,"/st-gaultier,d80621488",3,515,36800,a35d11ecbd41c74f961f2423f7b9218c,36
433759,433782.0,6.02,"36800 Saint-gaultier - Terrain, 515m2",8189.0,2020-04-05,"/st-gaultier,d79708744",3,515,36800,3859f66aecd90324b577a391d3e9c3db,36
446470,446493.0,-15.23,"36800 Saint-gaultier - Terrain, 515m2",8189.0,2020-04-20,"/st-gaultier,d79968279",3,515,36800,ddc7803dcc2b083ce06c1cb9639f9213,36
450255,450278.0,-3.09,"36800 Saint-gaultier - Terrain, 515m2",8189.0,2020-04-25,"/st-gaultier,d79968279",3,515,36800,ddc7803dcc2b083ce06c1cb9639f9213,36
440120,440143.0,-10.92,"36800 Saint-gaultier - Terrain, 515m2",8189.0,2020-04-11,"/st-gaultier,d79708744",3,515,36800,3859f66aecd90324b577a391d3e9c3db,36
436808,436831.0,20.16,"36800 Saint-gaultier - Terrain, 515m2",8189.0,2020-04-07,"/st-gaultier,d79708744",3,515,36800,3859f66aecd90324b577a391d3e9c3db,36
455775,455798.0,-1.41,"36800 Saint-gaultier - Terrain, 515m2",8189.0,2020-05-01,"/st-gaultier,d79968279",3,515,36800,ddc7803dcc2b083ce06c1cb9639f9213,36
428429,428452.0,-0.06,"36800 Saint-gaultier - Terrain, 515m2",8189.0,2020-03-31,"/st-gaultier,d63859825",3,515,36800,1b3b10d593d1df1afee0b21db5e7f508,36


In [ ]:
df['combinaison'] = [[x, y,z] for x, y,z in zip(df.prix, df.date,df.taux)] #on cree une colonne avec trois elements

In [5]:
df=itemizer(df)

In [6]:
def sortSecond(val): #on fait en sor que les valeur ssoient bien trièées par date de plus vieux au plus récent
    return val[1]  
def sorter(list):
    list.sort(key = sortSecond) 
    return(list)
df['combinaison'] = df['combinaison'].apply(sorter)

In [7]:
#def tri(list):
def variation(list):
    y=0
    for x in list:
        y=y+x[2]
    return(y)
def lastchangedate(list):
    return(list[len(list)-1][1])
def lastchangedate2(list):
    return(list[len(list)-2][1])
def lastchangevar(list):
    return(list[len(list)-1][2])
def lastchangevar2(list):
    return(list[len(list)-2][2])

df['variation'] = df['combinaison'].apply(variation)
df['lastchangedate'] = df['combinaison'].apply(lastchangedate)
df['lastchangedate2'] = df['combinaison'].apply(lastchangedate2)
df['lastchangevar'] = df['combinaison'].apply(lastchangevar)
df['lastchangevar2'] = df['combinaison'].apply(lastchangevar2)


In [8]:
xf=df[df.lastchangedate > datetime.datetime.now() - pd.to_timedelta("30day")]

In [9]:

xf.tail(10)

,hash,combinaison,variation,lastchangedate,lastchangedate2,lastchangevar,lastchangevar2
39817,ff8219c72774e8b1f396a29dfd5bf3c4,"[[352000.0, 2019-11-01 00:00:00, -0.85], [3350...",-6.48,2020-05-09,2019-11-01,-5.63,-0.85
39823,ff8fedb90d786eafcd5909c92f66d515,"[[265000.0, 2020-03-06 00:00:00, -3.64], [2600...",-9.09,2020-05-20,2020-03-06,-5.45,-3.64
39828,ff99e81ad87cc8e4e1dcf31d4eb02ed6,"[[209000.0, 2020-03-16 00:00:00, -3.69], [1980...",-12.45,2020-05-16,2020-03-16,-8.76,-3.69
39842,ffb5b0259169a53a54c446bb26e9c975,"[[219000.0, 2019-11-10 00:00:00, -6.81], [2150...",-15.32,2020-04-25,2019-11-10,-8.51,-6.81
39843,ffb657fa008d62957afda5acfc984bdd,"[[257340.0, 2019-11-08 00:00:00, 0.88], [22027...",-26.61,2020-05-16,2020-02-26,-13.84,-13.65
39847,ffbac59e67d5a30e2e0227fad7f8e2c7,"[[63173.0, 2019-12-22 00:00:00, -2.94], [63173...",-22.06,2020-04-25,2020-04-17,-8.40,3.64
39856,ffcab705b1f0c9f84b22558a994fda5f,"[[408000.0, 2020-04-05 00:00:00, 0.74], [42100...",4.69,2020-05-17,2020-04-05,3.95,0.74
39861,ffd0d8669e6778b1b745b28d89c661da,"[[130123.0, 2020-03-14 00:00:00, -37.25], [130...",-223.50,2020-05-21,2020-05-16,-37.25,-37.25
39875,ffe495aa666d20bddb801c0f10be0ba8,"[[315000.0, 2020-02-19 00:00:00, -0.94], [3050...",-12.26,2020-05-18,2020-03-14,-7.23,-4.09
39885,fffcb9530758b519cf6cbc8541725321,"[[147964.0, 2020-04-03 00:00:00, 2.0], [153344...",15.27,2020-05-15,2020-04-26,7.56,5.71
